In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numba
import math
import multiprocessing
import time
import argparse

%matplotlib qt5
os.getcwd()

'C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform'

In [22]:
data = ['100, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786','1000, 28015119.5, 1, 3.56, 24719.24, 12.4, 843.2, 0.00000002, 23.82, 72271','101, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786']
sorted(data, key=lambda s: int(s.split(',')[0]))

['100, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786',
 '101, 12012515.3, 1, 3.56, 29602.05, 28.0, 844.6, 0.00000002, 41.10, 707786',
 '1000, 28015119.5, 1, 3.56, 24719.24, 12.4, 843.2, 0.00000002, 23.82, 72271']

In [ ]:
s=time.time()
np.sum(pow(valid_data, 2))
e=time.time()
print(e-s)

In [ ]:
class Preprocessing:
    def __init__(self, file_name, thr_dB):
        self.file_name = file_name
        self.thr_V = float('%.8f' % (pow(10, thr_dB / 20) / 5000000))
        self.counts = 1
        self.duration = 0
        self.amplitude = 0
        self.rise_time = 0
        self.energy = 0

    def skip_n_column(self, file, n=3):
        for _ in range(n):
            file.readline()

    def cal_features(self, dataset, time_label):
        valid_wave_idx = np.where(abs(dataset) >= self.thr_V)[0]
        start = time_label[valid_wave_idx[0]]
        end = time_label[valid_wave_idx[-1]]
        self.duration = end - start
        max_idx = np.argmax(abs(dataset))
        self.amplitude = dataset[max_idx]
        self.rise_time = time_label[max_idx] - start
        valid_data = dataset[valid_wave_idx[0]:valid_wave_idx[-1]]
        self.energy = np.sum(pow(valid_data, 2))

        return valid_data

    def cal_counts(self, valid_data):
        N = len(valid_data)
        for idx in range(1, N):
            if valid_data[idx - 1] <= self.thr_V <= valid_data[idx] \
                or valid_data[idx] <= self.thr_V <= valid_data[idx - 1]:
                self.counts += 1

    def main(self):
        for name in tqdm(self.file_name):
            with open(name, "r") as f:
                self.skip_n_column(f)
                sample_interval = float(f.readline()[29:])
                self.skip_n_column(f)
                points_num = int(f.readline()[36:])
                channel_num = int(f.readline().strip()[16:])
                hit_num = int(f.readline()[12:])
                time = float(f.readline()[14:])
                dataset = np.array([float(i.strip("\n")) for i in f.readlines()[1:]])
                time_label = np.linspace(time, time + sample_interval * (points_num - 1), points_num)

                # calculate the duration, amplitude, rise_time, energy and counts
                valid_data = self.cal_features(dataset, time_label)
                self.cal_counts(valid_data)
#                 print(name.split('_')[2], sample_interval, points_num, channel_num, hit_num, time)
#                 print('-' * 50)
            print(self.counts, self.duration, self.amplitude, self.rise_time, self.energy)
#             return self.counts, self.duration, self.amplitude, self.rise_time, self.energy

In [7]:
path = 'C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform'
os.chdir(path)
file_list = os.listdir(path)
file_list[:1]

['CM-6M-o18-2020.10.17-1-60_1_100000_967649009.txt']

In [ ]:
a = Preprocessing(file_list[:100], 25)
a.main()

In [ ]:
path = 'C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform'
os.chdir(path)
file_list = os.listdir(path)
thr_dB = 25
thr_V = float('%.8f' % (pow(10, thr_dB / 20) / 5000000))
core_num = 4
each_core = int(math.ceil(len(file_list) / float(4)))  # 每一个核心需要处理的list的数目



# name = 'CM-6M-o18-2020.10.17-1-60_1_100_12012515.txt'

# for name in tqdm(file_list):


In [ ]:
if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--folder_path", type=str, default="C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform", help="path to dataset")
#     parser.add_argument("--thr_dB", type=int, default=25, help="the detection threshold")
#     parser.add_argument("--threads", type=int, default=4, help="the detection threshold")
#     opt = parser.parse_args()
#     print(opt)
    folder_path = "C:\\Users\\Yuan\\Desktop\\CM-6M-o18-2020.10.17-1-60-a-waveform"
    threads = 4
    thr_dB = 25
#     os.chdir(path)
    file_list = os.listdir(folder_path)
    each_core = int(math.ceil(len(file_list) / float(4)))
    result = []
    # Multithreading acceleration
    pool = multiprocessing.Pool(processes=threads)
    for i in tqdm(range(0, len(file_list), each_core)):
        result.append(pool.apply_async(Preprocessing(file_list[i:i + each_core], thr_dB), (file_list[i:i + each_core], )))
    pool.close()
    pool.join()
    